Lab: Debugging Mutable Default Arguments with Assert Statements
Overview
In this lab, you will:

Add assert statements to verify assumptions in the create_picnic_basket function.

Understand the root cause of the issue: mutable default arguments.

Correct the error by using an immutable default value and applying the proper initialization.

Notice that the third basket unexpectedly includes items from the first basket because the DEFAULT_INITIAL_BASKET object was modified. This happens because mutable default arguments in Python persist across function calls, causing unintended side effects.

In [2]:
DEFAULT_INITIAL_BASKET = ["orange", "apple"]

def create_picnic_basket(healthy, hungry, initial_basket=DEFAULT_INITIAL_BASKET):
    basket = initial_basket
    if healthy:
        basket.append("strawberry")
    else:
        basket.append("jam")

    if hungry:
        basket.append("sandwich")
    return basket

# Reproducer
print("First basket:", create_picnic_basket(True, False))
print("Second basket:", create_picnic_basket(False, True, ["tea"]))
print("Third basket:", create_picnic_basket(True, True))


First basket: ['orange', 'apple', 'strawberry']
Second basket: ['tea', 'jam', 'sandwich']
Third basket: ['orange', 'apple', 'strawberry', 'strawberry', 'sandwich']


In [1]:
#need to correct assertions


DEFAULT_INITIAL_BASKET = ["orange", "apple"]

def create_picnic_basket(healthy, hungry, initial_basket=DEFAULT_INITIAL_BASKET):
    # Assert that the default basket is unchanged at the start of each function call
    assert isinstance(initial_basket, list), "initial_basket must be a list"
    assert "orange" in DEFAULT_INITIAL_BASKET and "apple" in DEFAULT_INITIAL_BASKET, (
        f"DEFAULT_INITIAL_BASKET has been modified: {DEFAULT_INITIAL_BASKET}"
    )

    # Assign the basket (potentially modifies the global DEFAULT_INITIAL_BASKET)
    basket = initial_basket

    # Verify that the healthy condition is applied correctly
    if healthy:
        basket.append("strawberry")
        assert "strawberry" in basket, "Failed to add 'strawberry' for healthy basket"
    else:
        basket.append("jam")
        assert "jam" in basket, "Failed to add 'jam' for unhealthy basket"

    # Verify that the hungry condition is applied correctly
    if hungry:
        basket.append("sandwich")
        assert "sandwich" in basket, "Failed to add 'sandwich' for hungry basket"

    # Return the basket
    return basket

# Reproducer
print("First basket:", create_picnic_basket(True, False))   # Expect: ['orange', 'apple', 'strawberry']
print("Second basket:", create_picnic_basket(False, True, ["tea"]))  # Expect: ['tea', 'jam', 'sandwich']
print("Third basket:", create_picnic_basket(True, True))    # Expect: ['orange', 'apple', 'strawberry', 'sandwich']


First basket: ['orange', 'apple', 'strawberry']
Second basket: ['tea', 'jam', 'sandwich']
Third basket: ['orange', 'apple', 'strawberry', 'strawberry', 'sandwich']


# Correcting the Error
The root cause is the use of a mutable default argument (DEFAULT_INITIAL_BASKET). When the basket is modified, it affects the global default object. To fix this, we can use an immutable default value (None) and initialize a new basket inside the function if no basket is provided.

In [2]:
#corrected code
DEFAULT_INITIAL_BASKET = ["orange", "apple"]

def create_picnic_basket(healthy, hungry, initial_basket=None):
    # Use a new copy of the default basket if no initial_basket is provided
    if initial_basket is None:
        basket = DEFAULT_INITIAL_BASKET.copy()
    else:
        basket = initial_basket

    # Verify that the healthy condition is applied correctly
    if healthy:
        basket.append("strawberry")
    else:
        basket.append("jam")

    # Verify that the hungry condition is applied correctly
    if hungry:
        basket.append("sandwich")

    return basket

# Reproducer
print("First basket:", create_picnic_basket(True, False))   # Expect: ['orange', 'apple', 'strawberry']
print("Second basket:", create_picnic_basket(False, True, ["tea"]))  # Expect: ['tea', 'jam', 'sandwich']
print("Third basket:", create_picnic_basket(True, True))    # Expect: ['orange', 'apple', 'strawberry', 'sandwich']


First basket: ['orange', 'apple', 'strawberry']
Second basket: ['tea', 'jam', 'sandwich']
Third basket: ['orange', 'apple', 'strawberry', 'sandwich']


In [5]:
DEFAULT_INITIAL_BASKET = ["orange", "apple"]
EXPECTED_DEFAULT = ["orange", "apple"]  # Our expected starting basket

def create_picnic_basket(healthy, hungry, initial_basket=DEFAULT_INITIAL_BASKET):
    # If the user did not provide a basket and we are using the default one,
    # assert that it has not been altered from its expected value.
    if initial_basket is DEFAULT_INITIAL_BASKET:
        assert initial_basket == EXPECTED_DEFAULT, (
            f"Error: DEFAULT_INITIAL_BASKET has been modified: {initial_basket}"
        )
    
    basket = initial_basket
    if healthy:
        basket.append("strawberry")
    else:
        basket.append("jam")

    if hungry:
        basket.append("sandwich")
    return basket

# Reproducer calls:
print("First basket:", create_picnic_basket(True, False))
print("Second basket:", create_picnic_basket(False, True, ["tea"]))
# The third call uses the default basket again.
# At this point, because the first call modified DEFAULT_INITIAL_BASKET,
# the assertion below should trigger.
print("Third basket:", create_picnic_basket(True, True))


First basket: ['orange', 'apple', 'strawberry']
Second basket: ['tea', 'jam', 'sandwich']


AssertionError: Error: DEFAULT_INITIAL_BASKET has been modified: ['orange', 'apple', 'strawberry']